# Drivers of $\Omega_{arag}$ extremes in SMYLE FOSI:
(1) look at the change in CO3 with varying DIC, Alk, T, S - what is dominant?

    (a) Define relationships between these variables and CO3
    (b) Isolate CO3 to periods when $\Omega_{arag}$ is extreme
    (c) Calculate the relative change to CO3 from each variable - what is the largest contributor?

(2) If DIC is dominant, look at the tendency terms during events

In [ ]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import copy
import scipy.stats
from scipy import signal
from functools import partial
import glob
import dask
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import esmtools as esmtools

# ! pip install PyCO2SYS
import PyCO2SYS as pyco2

## (1) Use PyCO2SYS to determine the relationships

### (a)

In [ ]:
var = "CO3"
depth = "surface"
time = "monthly"

# open the data!
ds = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/'+ var +'.' + time + '.' + depth + '.regrid.nc')[var]
ds['time'] = pd.date_range("1958-01", "2020-12", freq="MS")

In [ ]:
var = "TEMP"
depth = "surface"
time = "monthly"
# open the data!
ds_t = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/'+ var +'.' + time + '.' + depth + '.regrid.nc')[var]
ds_t['time'] = pd.date_range("1958-01", "2020-12", freq="MS")

In [ ]:
var = "SALT"
depth = "surface"
time = "monthly"
# open the data!
ds_s = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/'+ var +'.' + time + '.' + depth + '.regrid.nc')[var].isel(z_t=0)
ds_s['time'] = pd.date_range("1958-01", "2020-12", freq="MS")

In [ ]:
var = "DIC"
depth = "surface"
time = "monthly"
# open the data!
ds_dic = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/'+ var +'.' + time + '.' + depth + '.regrid.nc')[var]
ds_dic['time'] = pd.date_range("1958-01", "2020-12", freq="MS")

In [ ]:
var = "ALK"
depth = "surface"
time = "monthly"
# open the data!
ds_alk = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/'+ var +'.' + time + '.' + depth + '.regrid.nc')[var].isel(z_t=0)
ds_alk['time'] = pd.date_range("1958-01", "2020-12", freq="MS")

In [ ]:
# set up background values for Alkalinity, DIC, Salinity
ALK = 2280
DIC = 2040
SAL = 34
T = 20

In [ ]:
# T
T_range = np.arange(0, 30, 0.1)
s = T_range.size;
tmp = []

for i in range (1,s):
    A = pyco2.CO2SYS_nd(ALK,DIC,1,2,SAL,T_range[i],T_range[i],5,0,1,10,1);
    tmp.append(A['carbonate'])

pH_sst = np.array(tmp)

from scipy.stats import linregress

m, b, r, p, err = linregress(T_range[0:299],pH_sst) 

print(m,b)

vals_t = m * T_range + b

ds_t_signal = ds_t * m + b
ds_t_residual = ds - ds_t_signal

In [ ]:
plt.plot(vals_t, T_range)

In [ ]:
# S
s_range = np.arange(28, 38, 0.1)
tmp = []

for i in range (1,s_range.size):
    A = pyco2.CO2SYS_nd(ALK,DIC,1,2,s_range[i],T,T,5,0,1,10,1);
    tmp.append(A['carbonate'])

pH_sst = np.array(tmp)

from scipy.stats import linregress

m, b, r, p, err = linregress(s_range[0:-1],pH_sst) 

print(m,b)

vals_t = m * s_range + b

ds_s_signal = ds_s * m + b
ds_s_residual = ds - ds_s_signal

In [ ]:
plt.plot(vals_t, s_range)

In [ ]:
# S
dic_range = np.arange(1800, 2500, 1)
tmp = []

for i in range (1,dic_range.size):
    A = pyco2.CO2SYS_nd(ALK,dic_range[i],1,2,SAL,T,T,5,0,1,10,1);
    tmp.append(A['carbonate'])

pH_sst = np.array(tmp)

from scipy.stats import linregress

m, b, r, p, err = linregress(dic_range[0:-1],pH_sst) 

print(m,b)

vals_t = m * dic_range + b

ds_dic_signal = ds_dic * m + b
ds_dic_residual = ds - ds_dic_signal

In [ ]:
plt.plot(vals_t, dic_range)

In [ ]:
# S
alk_range = np.arange(1800, 2500, 1)
tmp = []

for i in range (1,alk_range.size):
    A = pyco2.CO2SYS_nd(alk_range[i],DIC,1,2,SAL,T,T,5,0,1,10,1);
    tmp.append(A['carbonate'])

pH_sst = np.array(tmp)

from scipy.stats import linregress

m, b, r, p, err = linregress(alk_range[0:-1],pH_sst) 

print(m,b)

vals_t = m * alk_range + b

ds_alk_signal = ds_alk * m + b
ds_alk_residual = ds - ds_alk_signal

In [ ]:
plt.plot(vals_t, alk_range)

### (b) isolate these to where $\Omega_{arag}$ is extreme

In [ ]:
extreme = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/omega_arag.fosi.rolling.thold.Rolling.nc')['threshold']
extreme = extreme.where(extreme.lat < 65)

In [ ]:
extreme.isel(time=0).plot()

In [ ]:
ds_ext = ds.where(extreme == 1)

In [ ]:
ds_t_signal_ext = ds_t_signal.where(extreme == 1)

In [ ]:
ds_s_signal_ext = ds_s_signal.where(extreme == 1)

In [ ]:
ds_dic_signal_ext = ds_dic_signal.where(extreme == 1)

In [ ]:
ds_alk_signal_ext = ds_alk_signal.where(extreme == 1)

### (c) the relative change from each variable

- high alk = high co3
- high dic = low co3
- high t = high co3
- high s = low co3

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

## so now we will look at the tendency terms

In [ ]:
# conversion =  1/1000 * 1/100 * 60*60*24 # mmol to mol ; cm to m ; s to day

In [ ]:
var = "DIC" # units = mmol/m^3
depth = "surface"
time = "monthly"
# open the data!
ds_dic = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/'+ var +'.' + time + '.' + depth + '.regrid.nc')[var]
ds_dic['time'] = pd.date_range("1958-01", "2020-12", freq="MS")

In [ ]:
var = "FG_CO2" # units = mmol/m^3 cm/s
depth = "surface"
time = "monthly"

# open the data!
fg_co2 = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/'+ var +'.' + time + '.' + depth + '.regrid.nc')[var]
fg_co2['time'] = pd.date_range("1958-01", "2020-12", freq="MS")
# fg_co2 = fg_co2 * conversion

In [ ]:
var = "Jint_100m_DIC" # units = mmol/m^3 cm/s
depth = "surface"
time = "monthly"

# open the data!
jint = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/'+ var +'.' + time + '.' + depth + '.regrid.nc')[var]
jint['time'] = pd.date_range("1958-01", "2020-12", freq="MS")
# jint = jint * conversion

In [ ]:
var = "tend_zint_100m_DIC" # units = mmol/m^3 cm/s
depth = "surface"
time = "monthly"

# open the data!
tend = xr.open_dataset('/glade/work/smogen/SMYLE-extremes/FOSI/'+ var +'.' + time + '.' + depth + '.regrid.nc')[var]
tend['time'] = pd.date_range("1958-01", "2020-12", freq="MS")
# tend = tend * conversion

In [ ]:
ds_dic = ds_dic.where(ds_dic > 0)

tend = tend.where(ds_dic > 0)
jint = jint.where(ds_dic > 0)
fg_co2 = fg_co2.where(ds_dic > 0)

In [ ]:
circ = (tend - jint - fg_co2)

In [ ]:
dic_ex = ds_dic.where(extreme == 1)

tend_ex = tend.where(extreme == 1)

circ_ex = circ.where(extreme == 1)
fg_co2_ex = fg_co2.where(extreme == 1)
jint_ex = jint.where(extreme == 1)

### Correlation

In [ ]:
tend_circ = xr.corr(tend, circ, dim='time')
tend_fg = xr.corr(tend, fg_co2, dim='time')
tend_jint = xr.corr(tend, jint, dim='time')

f, ax = plt.subplots(1,3, figsize = (9,2),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

tend_circ.plot(ax=ax[0],transform = ccrs.PlateCarree())
ax[0].set_title('$Circulation_{component}$')

tend_fg.plot(ax=ax[1],transform = ccrs.PlateCarree())
ax[1].set_title('$Air-Sea Flux_{component}$')

tend_jint.plot(ax=ax[2],transform = ccrs.PlateCarree())
ax[2].set_title('$Biological_{component}$')

f.suptitle('Correlation (r) of $DIC_{tendency}$ and drivers at all times')
plt.tight_layout()

In [ ]:
tend_circ = xr.corr(ds_dic, circ, dim='time')
tend_fg = xr.corr(ds_dic, fg_co2, dim='time')
tend_jint = xr.corr(ds_dic, jint, dim='time')

f, ax = plt.subplots(1,3, figsize = (9,2),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

tend_circ.plot(ax=ax[0],transform = ccrs.PlateCarree())
ax[0].set_title('$Circulation_{component}$')

tend_fg.plot(ax=ax[1],transform = ccrs.PlateCarree())
ax[1].set_title('$Air-Sea Flux_{component}$')

tend_jint.plot(ax=ax[2],transform = ccrs.PlateCarree())
ax[2].set_title('$Biological_{component}$')

f.suptitle('Correlation (r) of $DIC$ and drivers at all times')
plt.tight_layout()

In [ ]:
tend_circ = xr.corr(tend_ex, circ_ex, dim='time')
tend_fg = xr.corr(tend_ex, fg_co2_ex, dim='time')
tend_jint = xr.corr(tend_ex, jint_ex, dim='time')

f, ax = plt.subplots(1,3, figsize = (9,2),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

tend_circ.plot(ax=ax[0],transform = ccrs.PlateCarree())
ax[0].set_title('$Circulation_{component}$')

tend_fg.plot(ax=ax[1],transform = ccrs.PlateCarree())
ax[1].set_title('$Air-Sea Flux_{component}$')

tend_jint.plot(ax=ax[2],transform = ccrs.PlateCarree())
ax[2].set_title('$Biological_{component}$')

f.suptitle('Correlation (r) of $DIC_{tendency}$ and drivers during OAX ($\Omega_{arag}$)')
plt.tight_layout()

In [ ]:
dic_circ = xr.corr(dic_ex, circ_ex, dim='time')
dic_fg = xr.corr(dic_ex, fg_co2_ex, dim='time')
dic_jint = xr.corr(dic_ex, jint_ex, dim='time')

f, ax = plt.subplots(1,3, figsize = (9,2),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

dic_circ.plot(ax=ax[0],transform = ccrs.PlateCarree())
ax[0].set_title('$Circulation_{component}$')

dic_fg.plot(ax=ax[1],transform = ccrs.PlateCarree())
ax[1].set_title('$Air-Sea Flux_{component}$')

dic_jint.plot(ax=ax[2],transform = ccrs.PlateCarree())
ax[2].set_title('$Biological_{component}$')

f.suptitle('Correlation (r) of $DIC$ and drivers during OAX ($\Omega_{arag}$)')
plt.tight_layout()

## Magnitude of components

In [ ]:
f, ax = plt.subplots(1,1, subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))
ds_dic.mean('time').plot(ax=ax,transform = ccrs.PlateCarree(), cmap='Reds', vmin=1800, vmax = 2500)

In [ ]:
f, ax = plt.subplots(1,4, figsize = (12,2),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

tend.mean('time').plot(ax=ax[0],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[0].set_title('$Tendency$')

circ.mean('time').plot(ax=ax[1],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[1].set_title('$Circulation_{component}$')

fg_co2.mean('time').plot(ax=ax[2],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[2].set_title('$Air-Sea Flux_{component}$')

jint.mean('time').plot(ax=ax[3],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[3].set_title('$Biological_{component}$')

f.suptitle('Magnitude of $DIC$ drivers at all times')
plt.tight_layout()

In [ ]:
f, ax = plt.subplots(1,1, subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))
dic_ex.mean('time').plot(ax=ax,transform = ccrs.PlateCarree(), cmap='Reds', vmin=1800, vmax = 2500)

In [ ]:
f, ax = plt.subplots(1,4, figsize = (12,2),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

tend_ex.mean('time').plot(ax=ax[0],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[0].set_title('$Tendency$')

circ_ex.mean('time').plot(ax=ax[1],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[1].set_title('$Circulation_{component}$')

fg_co2_ex.mean('time').plot(ax=ax[2],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[2].set_title('$Air-Sea Flux_{component}$')

jint_ex.mean('time').plot(ax=ax[3],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[3].set_title('$Biological_{component}$')

f.suptitle('Magnitude of $DIC$ drivers during OAX ($\Omega_{arag}$)')
plt.tight_layout()

In [ ]:
#  change between normal and extremes
f, ax = plt.subplots(1,4, figsize = (12,2),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

(tend_ex.mean('time') - tend.mean('time')).plot(ax=ax[0],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[0].set_title('$Tendency$')

(circ_ex.mean('time') - circ.mean('time')).plot(ax=ax[1],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[1].set_title('$Circulation_{component}$')

(fg_co2_ex.mean('time') - fg_co2.mean('time')).plot(ax=ax[2],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[2].set_title('$Air-Sea Flux_{component}$')

(jint_ex.mean('time') - jint.mean('time')).plot(ax=ax[3],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm')
ax[3].set_title('$Biological_{component}$')

f.suptitle('($\Delta$) (or, how much more important is a term during OAX)')
plt.tight_layout()

## Paper Figure
 need to run everything above!

In [ ]:
# drivers of changes in CO3 during OAX relative to all times
f, ax = plt.subplots(1,4, figsize = (16,10),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

(ds_t_signal_ext.mean('time') - (ds_t_signal.where(ds_dic_signal_ext.mean('time') > 0)).mean('time')).plot(ax=ax[0],transform = ccrs.PlateCarree(), vmin=-5, vmax=5, cmap='coolwarm', extend='both',add_colorbar=False)
ax[0].set_title('$T_{component}$')
ax[0].add_feature(cfeature.LAND, color='k', zorder=3)

(ds_s_signal_ext.mean('time') - ds_s_signal.where(ds_dic_signal_ext.mean('time') > 0)).mean('time').plot(ax=ax[1],transform = ccrs.PlateCarree(), vmin=-5, vmax=5, cmap='coolwarm', extend='both',add_colorbar=False)
ax[1].set_title('$S_{component}$')
ax[1].add_feature(cfeature.LAND, color='k', zorder=3)

(ds_dic_signal_ext.mean('time') - ds_dic_signal.where(ds_dic_signal_ext.mean('time') > 0).mean('time')).plot(ax=ax[2],transform = ccrs.PlateCarree(), vmin=-5, vmax=5, cmap='coolwarm', extend='both',add_colorbar=False)
ax[2].set_title('$DIC_{component}$')
ax[2].add_feature(cfeature.LAND, color='k', zorder=3)

im = (ds_alk_signal_ext.where(ds_dic_signal_ext.mean('time') > 0) - ds_alk_signal.where(ds_dic_signal_ext.mean('time') > 0)).mean('time').plot(ax=ax[3],transform = ccrs.PlateCarree(), vmin=-5, vmax=5, cmap='coolwarm', extend='both',add_colorbar=False)
ax[3].set_title('$Alk_{component}$')
ax[3].add_feature(cfeature.LAND, color='k', zorder=3)

# f.suptitle('How much more does a given component of $CO_3$ matter during OAX')
plt.tight_layout()

f.subplots_adjust(right=0.8)
cbar_ax = f.add_axes([0.81, 0.425, 0.015, 0.15])
cbar = f.colorbar(im, cax=cbar_ax, ticks=[-5, 0, 5],extend='both')
cbar.ax.tick_params(labelsize=7)

f.savefig('./figures/CO3.drivers.R1.pdf')

In [ ]:
# drivers of changes in DIC during OAX relative to all times
f, ax = plt.subplots(1,4, figsize = (16,10),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

((tend_ex.mean('time') - tend.mean('time'))).plot(ax=ax[0],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm', extend='both',add_colorbar=False)
ax[0].set_title('$Tendency$')
ax[0].add_feature(cfeature.LAND, color='k', zorder=3)

(circ_ex.mean('time') - circ.mean('time')).plot(ax=ax[1],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm', extend='both',add_colorbar=False)
ax[1].set_title('$Circulation_{component}$')
ax[1].add_feature(cfeature.LAND, color='k', zorder=3)

(fg_co2_ex.mean('time') - fg_co2.mean('time')).plot(ax=ax[2],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm', extend='both',add_colorbar=False)
ax[2].set_title('$Air-Sea Flux_{component}$')
ax[2].add_feature(cfeature.LAND, color='k', zorder=3)

im = (jint_ex.mean('time') - jint.mean('time')).plot(ax=ax[3],transform = ccrs.PlateCarree(),vmin=-0.01,vmax=0.01,cmap='coolwarm', extend='both',add_colorbar=False)
ax[3].set_title('$Biological_{component}$')
ax[3].add_feature(cfeature.LAND, color='k', zorder=3)

# f.suptitle('How much more does a given component of $CO_3$ matter during OAX')
plt.tight_layout()

f.subplots_adjust(right=0.8)
cbar_ax = f.add_axes([0.81, 0.425, 0.015, 0.15])
cbar = f.colorbar(im, cax=cbar_ax, ticks=[-0.01, 0, 0.01],extend='both')
cbar.ax.tick_params(labelsize=7)

f.savefig('./figures/DIC.drivers.R1.pdf')

In [ ]:
# f, ax = plt.subplots(1,3, figsize = (12,10),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

# tend.where(tend.lat < 65).mean('time').plot(ax=ax[0],transform = ccrs.PlateCarree(),vmin=-0.005,vmax=0.005,cmap='coolwarm', extend='both',add_colorbar=False)
# ax[0].set_title('Mean $DIC_{Tendency}$')
# ax[0].add_feature(cfeature.LAND, color='k', zorder=3)

# tend_ex.mean('time').plot(ax=ax[1],transform = ccrs.PlateCarree(),vmin=-0.005,vmax=0.005,cmap='coolwarm', extend='both',add_colorbar=False)
# ax[1].set_title('Mean $DIC_{Tendency}$ (OAX)')
# ax[1].add_feature(cfeature.LAND, color='k', zorder=3)

# im = (-(tend.mean('time') - tend_ex.mean('time'))).plot(ax=ax[2],transform = ccrs.PlateCarree(),vmin=-0.005,vmax=0.005,cmap='coolwarm', extend='both',add_colorbar=False)
# ax[2].set_title('$\Delta_{Tendency}$')
# ax[2].add_feature(cfeature.LAND, color='k', zorder=3)

# plt.tight_layout()

# f.subplots_adjust(right=0.8)
# cbar_ax = f.add_axes([0.81, 0.425, 0.025, 0.15])
# cbar = f.colorbar(im, cax=cbar_ax, ticks=[-5e-3, 0, 0.005],extend='both')
# cbar.ax.tick_params(labelsize=7)
# # cbar.set_label('Forecast Skill Gain (SEDI)', rotation=270,fontsize=18)

# # plt.colorbar(im,extend='both')

In [ ]:
# f, ax = plt.subplots(1,3, figsize = (12,10),subplot_kw=dict(projection=ccrs.PlateCarree(central_longitude=180)))

# circ.where(tend.lat < 65).mean('time').plot(ax=ax[0],transform = ccrs.PlateCarree(),vmin=-0.005,vmax=0.005,cmap='coolwarm', extend='both',add_colorbar=False)
# ax[0].set_title('Mean $DIC_{circulation}$')
# ax[0].add_feature(cfeature.LAND, color='k', zorder=3)

# circ_ex.mean('time').plot(ax=ax[1],transform = ccrs.PlateCarree(),vmin=-0.005,vmax=0.005,cmap='coolwarm', extend='both',add_colorbar=False)
# ax[1].set_title('Mean $DIC_{circulation}$ (OAX)')
# ax[1].add_feature(cfeature.LAND, color='k', zorder=3)

# im = (-(circ.mean('time') - circ_ex.mean('time'))).plot(ax=ax[2],transform = ccrs.PlateCarree(),vmin=-0.005,vmax=0.005,cmap='coolwarm', extend='both',add_colorbar=False)
# ax[2].set_title('$\Delta_{circulation}$')
# ax[2].add_feature(cfeature.LAND, color='k', zorder=3)

# plt.tight_layout()

# f.subplots_adjust(right=0.8)
# cbar_ax = f.add_axes([0.81, 0.425, 0.025, 0.15])
# cbar = f.colorbar(im, cax=cbar_ax, ticks=[-5e-3, 0, 0.005],extend='both')
# cbar.ax.tick_params(labelsize=7)